In [1]:
# ПЕРЕД ПУСКОМ НУЖНО СНЕСТИ БД!

CLIENTS_AMOUNT = 200
ADVERTISERS_AMOUNT = 20
MAX_ADS_PER_ADVERTISER = 5
DAYS = 10

PROTO = "http"
BASE_URL = "127.0.0.1:8080"
NAME = "Main API"

In [2]:
import requests
from utils import *
import time
import random
from tqdm import tqdm

In [3]:
# Загрузить данные

locations = [
    "Moscow",
    "Samara",
    "Kazan",
    "St.Peterburg",
    "Syzran",
    "Irkutsk",
    "Tumen"
]

cl_genders = [
    "MALE",
    "FEMALE"
]

clients = [
    {
        "client_id": get_random_uuid(),
        "login": f"client_{i+1}",
        "age": random.randint(5, 60),
        "location": random.choice(locations),
        "gender": random.choice(cl_genders)
    }
    for i in range(CLIENTS_AMOUNT)
]

advertisers = [
    {
        "advertiser_id": get_random_uuid(),
        "name": f"advertiser_{i+1}"
    }
    for i in range(ADVERTISERS_AMOUNT)
]

ml_scores = [

]

for i in clients:
    for j in advertisers:
        ml_scores.append(
            {
                "client_id": i["client_id"],
                "advertiser_id": j["advertiser_id"],
                "score": random.randint(0, 100000)
            }
        )

campaigns = {}

for i in advertisers:
    campaigns[i['advertiser_id']] = []
    for j in range(random.randint(1, MAX_ADS_PER_ADVERTISER)):
        start_date = random.randint(0, DAYS)
        end_date = start_date + random.randint(0, DAYS - start_date)
        impressions_limit = random.randint(0, CLIENTS_AMOUNT)
        clicks_limit = int(impressions_limit * random.randint(50, 100) / 100) # от 50 до 100 % показов
        impressions_cost = random.randint(100, 1000) / 100 # от рубля до 10 рублей
        clicks_cost = int(impressions_cost * random.randint(150, 200) / 100) # от 150 до 200 % цены показа
        targeting = {

        }
        t_gender = random.choice(["MALE", "FEMALE", "ALL"])
        t_age_from = random.randint(0, 20)
        t_age_to = t_age_from + random.randint(0, 40)
        t_location = random.choice(locations)
        if random.randint(0, 100) > 70: targeting["gender"] = t_gender
        if random.randint(0, 100) > 70: targeting["age_from"] = t_age_from
        if random.randint(0, 100) > 70: targeting["age_to"] = t_age_to
        if random.randint(0, 100) > 70: targeting["location"] = t_location
        ad = {
            "impressions_limit": impressions_limit,
            "clicks_limit": clicks_limit,
            "cost_per_impression": impressions_cost,
            "cost_per_click": clicks_cost,
            "ad_title": "AD",
            "ad_text": "AD",
            "start_date": start_date,
            "end_date": end_date,
            "targeting": targeting
        }

        campaigns[i['advertiser_id']].append(ad)

print("data prepared")

start_time = time.time() 
requests.post(f"{PROTO}://{BASE_URL}/clients/bulk", json=clients)
print("loaded clients")
requests.post(f"{PROTO}://{BASE_URL}/advertisers/bulk", json=advertisers)
print("loaded advertisers")

for i in tqdm(range(len(ml_scores))):
    requests.post(f"{PROTO}://{BASE_URL}/ml-scores", json=ml_scores[i])

print("loaded ml scores")
for i in campaigns:
    for j in campaigns[i]:
        requests.post(f"{PROTO}://{BASE_URL}/advertisers/{i}/campaigns", json=j)
print("loaded campaigns")
end_time = time.time()

print(f"data loaded ({round((end_time - start_time) * 1000)}ms)")

data prepared
loaded clients
loaded advertisers


100%|██████████| 4000/4000 [00:46<00:00, 86.64it/s]


loaded ml scores
loaded campaigns
data loaded (47398ms)


In [4]:
# Симуляция активности
from functools import cache

@cache
def get_relative_score(client, adv_id):
    max_ = 0
    min_ = 1000000
    adv_score = 0
    for i in ml_scores:
        if i["client_id"] == client:
            if i["score"] > max_: max_ = i["score"]
            if i["score"] < min_: min_ = i["score"]
            if i["advertiser_id"] == adv_id: adv_score = i["score"]
    try:
        return (adv_score - min_) / (max_ - min_)
    except Exception as e:
        return 0

impressions = {}
clicks = {}

scores_sum = 0
all_imprs = 0
all_hits = 0
fails = 0

for i in clients:
    impressions[i['client_id']] = [ [] for _ in range(DAYS) ]
    clicks[i['client_id']] = [ [] for _ in range(DAYS) ]

for date in range(DAYS):
    print(f"День {date}")
    requests.post(f"{PROTO}://{BASE_URL}/time/advance", json={"current_date": date})

    views_today = random.randint(0, CLIENTS_AMOUNT * MAX_ADS_PER_ADVERTISER)

    all_hits += views_today

    for i in tqdm(range(views_today)):
        cl_id = random.choice(clients)['client_id']

        adr = requests.get(f"{PROTO}://{BASE_URL}/ads", params={"client_id": cl_id})
        ad = adr.json()
        try:
            ad["message"]
            fails += 1
        except Exception as e:
            ad_id = ad["ad_id"]
            adv_id = ad["advertiser_id"]

            impressions[cl_id][date].append(ad_id)
            
            r_score = get_relative_score(cl_id, adv_id)
            scores_sum += r_score
            all_imprs += 1

            if random.uniform(0.0,1.0) < r_score:
                requests.post(f"{PROTO}://{BASE_URL}/ads/{ad_id}/click", json={"client_id": cl_id})
                clicks[cl_id][date].append(ad_id)

День 0


100%|██████████| 554/554 [00:09<00:00, 60.61it/s]


День 1


100%|██████████| 573/573 [00:10<00:00, 55.83it/s]


День 2


...

100%|██████████| 259/259 [00:05<00:00, 44.65it/s]


День 8


100%|██████████| 491/491 [00:10<00:00, 45.57it/s]


День 9


100%|██████████| 404/404 [00:09<00:00, 44.55it/s]


In [5]:
allclicks = sum([ sum( len(j) for j in clicks[i] ) for i in clicks ])

print("Не удалось найти рекламу %:", fails / all_hits * 100)
print("Средняя релевантность %:", scores_sum / all_imprs * 100)
print("Конверсия (не статистика) %:", allclicks / all_imprs * 100)

Не удалось найти рекламу %: 11.069455406471981
Средняя релевантность %: 54.566053733039645
Конверсия (не статистика) %: 55.60239627246505


In [6]:
allimpr = 0
allclk = 0
all_spent = 0
for i in advertisers:
    d = requests.get(f"{PROTO}://{BASE_URL}/stats/advertisers/{i['advertiser_id']}/campaigns").json()
    print(d)
    allimpr += d["impressions_count"]
    allclk += d["clicks_count"]
    all_spent += d["spent_total"]

print("Всего просмотров (статистика): ", all_imprs)
print("Всего кликов (статистика): ", allclk)
print("Конверсия (статистика, глобально)", allclk / allimpr * 100)
print("Всего заработанно", all_spent)

{'impressions_count': 112, 'clicks_count': 66, 'conversion': 58.92857142857143, 'spent_impressions': 666.93, 'spent_clicks': 603.0, 'spent_total': 1269.9299999999998}


{'impressions_count': 173, 'clicks_count': 120, 'conversion': 69.36416184971098, 'spent_impressions': 1322.7699999999982, 'spent_clicks': 1508.0, 'spent_total': 2830.769999999998}
{'impressions_count': 226, 'clicks_count': 149, 'conversion': 65.929203539823, 'spent_impressions': 1612.6100000000026, 'spent_clicks': 1747.0, 'spent_total': 3359.6100000000024}
{'impressions_count': 167, 'clicks_count': 100, 'conversion': 59.88023952095808, 'spent_impressions': 1569.8000000000036, 'spent_clicks': 1700.0, 'spent_total': 3269.800000000004}
{'impressions_count': 0, 'clicks_count': 0, 'conversion': 0, 'spent_impressions': 0.0, 'spent_clicks': 0.0, 'spent_total': 0.0}
{'impressions_count': 0, 'clicks_count': 0, 'conversion': 0, 'spent_impressions': 0.0, 'spent_clicks': 0.0, 'spent_total': 0.0}
{'impressions_count': 165, 'clicks_count': 117, 'conversion': 70.9090909090909, 'spent_impressions': 1275.5799999999992, 'spent_clicks': 1377.0, 'spent_total': 2652.579999999999}
{'impressions_count': 143,